# Full Image Registration

### Spatial Mapping of Structure sets and Dose Extraction
- This notebook contains the code that performs and image registration on a planning and recurrence scan 
- it then finds the associated structures of interest and determines their spatial location 
- the original images are restricted to the spatial domain of the structure set of interest 
- deformable image registration is then applied to the cropped images 
- some code is then used to mask the transformed structure sets and extract the planning dose from within them 

In [ ]:
#import all libraries 
import os  
import shutil
import subprocess
import sys
import numpy
import fileinput
import re
import fnmatch
import shapely
import nibabel
import voxtox
import voxtox.image
import voxtox.utility
import time 



from shapely import geometry
from voxtox.rtstruct import getStructDoseDict
from voxtox.utility import getStructDict, pointIjkToXyz, getDatedObject
from quickviewer import QuickViewer
from quickviewer.viewer import write_translation_to_file
from thyroid_functions import *

In [ ]:
#set up all path names 
work_dir = os.path.join(os.getcwd(), 'tmp')
if not os.path.isdir(work_dir):
        os.makedirs(work_dir)

fixed_image='planning'
moving_image='relapse'
data_type = 'voxtox'
id_list = ['VT1_H_E09B41K1', 'VT1_H_5C3561K1', 'VT1_H_169B43K1', 'VT1_H_0326E2K1', 'VT1_H_F6A233K1']
patient_id = 'VT1_H_F6A233K1'
data_dir = '/Users/ceilidhwelsh/Documents/CambridgePhD/VoxtoxData/Relapse_Data/relapse'
patient_dir = f'{work_dir}/{patient_id}'
if not os.path.isdir(patient_dir):
        os.makedirs(patient_dir)
patient_path = f'{data_dir}/{patient_id}'
patient = voxtox.utility.Patient(patient_path)

roi_dict = {}
roi_dict["Thyroid"] = ["thyroid cartilage", "thyroid", "thyroid_cartilage", "Thyroid_cartilage_IG", \
                       "Thyroid cartilage IG", "thyroid_cartilage_IG", "thyroid_cartilage", \
                       "thyroid_cartilage_IG", "thyroid_cartliage", "thyroid cartilage", "thyroid cartilage IG",
                       "thyroid cartliage"]   

#roi_dict["LR"] = ["GCB_recurrence_LN", "Recurrence_Outline", \
#                    "Recurrence_GCB", "L_nodal_rec", "LR_GCB", "Recurrence Outline",
#                   "L nodal rec", "GCB recurrence_LN", "LR GCB", "Recurrence GCB", "L nodal_rec"]

#roi_dict["GTV"] = ["GTV", 'gvt', 'gtv', 'gvt65sjj']
        
studies = {}
ct = {}
nii_paths = {}
for case in [fixed_image, moving_image]:
    studies[case], ct[case] = get_ct_and_study(patient, case, data_type)
    nii_paths[case] = f'{patient_dir}/ct_{case}.nii'
    convert_dcm_to_nii(ct[case].path, nii_paths[case])
planning_scan = ct["planning"]

In [ ]:
roi_dict

In [ ]:
nii_fixed = nii_paths[fixed_image] 
nii_moving =  nii_paths[moving_image]
QuickViewer([nii_fixed, nii_moving])

In [ ]:
#functions for executing the elastix/transformix transformations
def apply_deformable_transform(fixed, moving, registration_dir="", parameter_file="", 
                               translation_file="", translation=False):
    
    if not os.path.isdir(registration_dir):
        os.makedirs(registration_dir)
    
    print("This is the parameter file in the function", parameter_file)
    cmd = ['elastix']
    cmd.extend(['-f', fixed])
    cmd.extend(['-m', moving])
    cmd.extend(['-out', f'{registration_dir}'])
    cmd.extend(['-p', f'{parameter_file}'])
    if translation:
        cmd.extend(['-t0', f'{translation_file}'])
        #mask_filenames = create_masks(translation_file=translation_file,
                                     #registration_dir=registration_dir)
        #cmd.extend(['-fMask', f'{mask_filenames[fixed_image]}'])
        #cmd.extend(['-mMask', f'{mask_filenames[moving_image]}'])
    print("> Final cmd call is: \n", cmd)
    subprocess.call(cmd)
    
    
    
    
def apply_rigid_transform(to_transform="", transformed_output_dir="",
                          transform_file="", transform_type='in'):
    print(transform_type)
    if not os.path.isdir(transformed_output_dir):
        os.makedirs(transformed_output_dir)

    cmd = f'transformix {transform_type} {to_transform} -out {transformed_output_dir} -tp {transform_file}'
    subprocess.run(cmd.split())
    
    if transform_type == '-def':
        transformed_file = f"{transformed_output_dir}/outputpoints.txt"
    else:
        transformed_file = f"{transformed_output_dir}/result.nii"
    
    print("> Final cmd call is:", cmd)
    
    return transformed_file
    

    
def create_masks(translation_file="", registration_dir=""):
    # Get z translation
    translation_vars = get_file_variables(translation_file)
    shift_z = -translation_vars['TransformParameters'][2]

    # Create masks
    image_fixed = nibabel.load(nii_paths[fixed_image])
    image_moving = nibabel.load(nii_paths[moving_image])
    masks = {}
    masks[fixed_image], masks[moving_image] = voxtox.image.create3DMasks(
        image_fixed, image_moving, shift_z=shift_z, dxy=50)
    mask_filenames = {}
    for case, mask in masks.items():
        mask_filenames[case] = f'{registration_dir}/mask_{case}.nii'
        mask.to_filename(mask_filenames[case])
    
    return mask_filenames
    
    
#function for replacing key words in transform files 
def replace_keywords(out_dir ="", 
                    search_keyword="", 
                    replace_keyword=""):
    for line in fileinput.input(out_dir, inplace=1):
        line = line.replace(search_keyword, replace_keyword)
        sys.stdout.write(line)
    
    
# Define the limits of the structure in the structure file 
def structure_bounds(input_image):
    x_struct = []
    y_struct = []
    z_struct = []
    
    struct_bounds = nibabel.load(input_image).get_fdata()
    for xid in range(struct_bounds.shape[0]):
        for i in struct_bounds[xid,:,:]:
            if numpy.any(i>0):
                x_struct.append(xid)
    x_shift = numpy.median(x_struct)
    x_struct = [min(x_struct), max(x_struct)]

    for yid in range(struct_bounds.shape[1]):
        for i in struct_bounds[:,yid,:]:
            if numpy.any(i>0):
                y_struct.append(yid)
    y_shift = numpy.median(y_struct)
    y_struct = [min(y_struct), max(y_struct)]

    for zid in range(struct_bounds.shape[2]):
        for i in struct_bounds[:,:,zid]:
            if numpy.any(i>0):
                z_struct.append(zid)
    z_shift = numpy.median(z_struct)
    z_struct = [min(z_struct), max(z_struct)]
    complete_struct_bounds = (x_struct, y_struct, z_struct)
    complete_shift = (x_shift, y_shift, z_shift)
    
    return complete_struct_bounds, complete_shift



# Find combination of the planning and relapse structure bounds to cover the coordinate space 
# of both thyroid outlines 
def adjust_structure_bounds(planning_struct_bounds, relapse_struct_bounds):
    """for the planning and relapse structure bounds find the minimum and maximum
    of the two coordinates in x and y. Return as the final structure bounds"""
    if planning_struct_bounds[0][0] <= relapse_struct_bounds[0][0]:
        x_struct = [planning_struct_bounds[0][0]]
    else:
        x_struct = [relapse_struct_bounds[0][0]]
    x_struct[0] = x_struct[0]
        
    if planning_struct_bounds[0][1] <= relapse_struct_bounds[0][1]:
        x_struct.append(relapse_struct_bounds[0][1])
    else:
        x_struct.append(planning_struct_bounds[0][1])
    x_struct[1] = x_struct[1]
    
    if planning_struct_bounds[1][0] <= relapse_struct_bounds[1][0]:
        y_struct = [planning_struct_bounds[1][0]]
    else:
        y_struct = [relapse_struct_bounds[1][0]]
    y_struct[0] = y_struct[0]
        
    if planning_struct_bounds[1][1] <= relapse_struct_bounds[1][1]:
        y_struct.append(relapse_struct_bounds[1][1])
    else:
        y_struct.append(planning_struct_bounds[1][1])
    y_struct[1] = y_struct[1]
        
    applied_struct_bounds = (x_struct, y_struct)
    
    return applied_struct_bounds



# Function to create the new image from the structure limits 
def create_bounded_image(original_image_path, structure_bounds=(), 
                        saved_image_path = ""):
    """pass the image to be bound and the associated structure bounds"""

    image_array = nibabel.load(original_image_path).get_fdata()
    image_affine = nibabel.load(original_image_path).affine
    print("this is the structure bounds in the function:", structure_bounds)
    print("inital image affine:", image_affine)
    print("to double check the image array shapes 0 and 1 ", image_array.shape[0], image_array.shape[0])
    image_affine[0,3] = image_affine[0,3] - (image_array.shape[0] - structure_bounds[0][1])*image_affine[1,1]
    image_affine[1,3] = image_affine[1,3] - (image_array.shape[1] - structure_bounds[1][1])*image_affine[0,0]
    #image_affine[2,3] = image_affine[2,3] + structure_bounds[2][0]*image_affine[2,2]
    print("edited image affine:", image_affine)
    
    image_array = image_array[structure_bounds[0][0]:structure_bounds[0][1],
                          structure_bounds[1][0]:structure_bounds[1][1], :]
    
    bounded_image = nibabel.Nifti1Image(image_array, image_affine)
    nibabel.nifti1.save(bounded_image, saved_image_path)
    
    return saved_image_path 

## Main Code 
The steps of the code are as follows:
- set up all the required files 
- find the shift required in x,y,z 
- apply the shift to the moving image scan and structure one after the other using transformix(?) 
- check and correct the affine matrices of the moving image transformed scans 
- find structure bounds and crop the fixed scan and the transformed moving scan 
- perform deformable registration on the cropped images 

In [ ]:
#Obtain the structures 
planning_structure_files = get_structure_file(patient_path, patient_dir,\
                    roi_dict, structure="planning", \
                    struct_names=roi_dict)
relapse_structure_files = get_structure_file(patient_path, patient_dir, \
                    roi_dict, structure="relapse", \
                    struct_names=roi_dict)  

In [ ]:
relapse_structure_files

In [ ]:
structure_files = {}
if fixed_image == 'planning':
    structure_files['fixed_image'] = planning_structure_files['Thyroid']
    structure_files['moving_image'] = relapse_structure_files['Thyroid']
else: 
    structure_files['fixed_image'] = relapse_structure_files['Thyroid']
    structure_files['moving_image'] = planning_structure_files['Thyroid']

In [ ]:
# Obtain the cropping bounds and shift values and store them 
planning_struct_bounds, planning_shift = structure_bounds(planning_structure_files['Thyroid'])
relapse_struct_bounds, relapse_shift = structure_bounds(relapse_structure_files['Thyroid'])
print("> The complete planning structure bounds are:", planning_struct_bounds, "\n")
print("> The complete relapse structure bounds are:", relapse_struct_bounds, "\n")
final_struct_bounds = adjust_structure_bounds(planning_struct_bounds, relapse_struct_bounds)
print("> The final structure bounds are:", final_struct_bounds, "\n")

superior_position = []    
superior_position.append(get_first_scan(nii_fixed))
superior_position.append(get_first_scan(nii_moving))
scan_z_shift = (superior_position[1]-superior_position[0])
y_shift = (planning_shift[1] - relapse_shift[1])
z_shift = (planning_shift[2] - relapse_shift[2])

if fixed_image == 'relapse':
    y_shift = -y_shift
    z_shift = -z_shift

print("thyroid z_shift vs original", z_shift, "vs", scan_z_shift, "\n")
final_z_shift = numpy.mean([scan_z_shift, z_shift])
print("> The y shift for the moving image is:", y_shift, "\n")
print("> The z shift for the moving image is:", final_z_shift, "\n")



In [ ]:
#Run first elastix registration
parameter_file = 'MI_Translation.txt'
parameter_files_dir = ''

registration = os.path.splitext(os.path.basename(parameter_file))[0]
translation_registration_dir = f'{patient_dir}/{fixed_image}_fixed/{registration}'
if not os.path.isdir(translation_registration_dir):
    os.makedirs(translation_registration_dir)

if parameter_files_dir == '':
    voxtox_dir = os.sep.join(voxtox.__file__.split(os.sep)[:-2])
    parameter_files_dir = f'{voxtox_dir}/examples/elastix/elastix_parameter_files/{parameter_file}'


apply_deformable_transform(fixed=nii_fixed, 
                           moving=nii_moving, 
                           registration_dir=translation_registration_dir, parameter_file=parameter_files_dir)

In [ ]:
# Write over the Parameters files with the new shift 
in_dir = f'{translation_registration_dir}/TransformParameters.0.txt'
out_dir = f'{translation_registration_dir}/TransformParameter_custom.txt'
write_translation_to_file(input_file=in_dir, output_file=out_dir,
                          dz=(final_z_shift), overwrite=True)
write_translation_to_file(input_file=out_dir, output_file=out_dir,
                          dy=(y_shift))

In [ ]:
# View initial registration performed by Elastix using Quickviewer 
QuickViewer([nii_fixed, f'{translation_registration_dir}/result.0.nii'], 
            title=["Planning Scan", "Relapse Scan"],
            figsize=5,
            orthog_view=True)

In [ ]:
# Apply a rigid transformation based on the overwritten parameter files 
apply_deformable_transform(fixed=nii_fixed,
                            moving=nii_moving,
                            registration_dir=translation_registration_dir, 
                            parameter_file=parameter_files_dir,
                            translation_file = out_dir, 
                            translation=True)

In [ ]:
# View the images after the new parameter files translation has been applied 
QuickViewer([nii_fixed, f'{translation_registration_dir}/result.0.nii'], zoom_ui=True,
           translation_file_to_overwrite=f'{translation_registration_dir}/TransformParameter_custom.txt', 
            translation=True, scale_in_mm=False, show_overlay=True)

## Change the slider to improve translation before running the next code block 

In [ ]:
#if changed the slides, re-run translation
apply_deformable_transform(fixed=nii_fixed, 
                           moving=nii_moving, 
                           registration_dir=translation_registration_dir, 
                           parameter_file=parameter_files_dir,
                           translation_file=out_dir,
                           translation=True)

QuickViewer([nii_fixed, f'{translation_registration_dir}/result.0.nii'], zoom_ui=True,
           translation_file_to_overwrite=f'{translation_registration_dir}/TransformParameter_custom.txt', 
            translation=True)

In [ ]:
# Transform structure set of interest using the final rigid transformation parameters
search_keyword = "FinalBSplineInterpolator"
replace_keyword = "FinalNearestNeighborInterpolator"
replace_keywords(out_dir=f'{translation_registration_dir}/TransformParameters.0.txt', 
                search_keyword=search_keyword, 
                replace_keyword=replace_keyword)

rigidly_transformed_structure = apply_rigid_transform(to_transform=structure_files['moving_image'], 
                        transformed_output_dir=f'{translation_registration_dir}/structure_transform_rigid',
                        transform_file=f'{translation_registration_dir}/TransformParameters.0.txt', 
                        transform_type='-in')


In [ ]:
# check the transform of the structure 
QuickViewer([f'{translation_registration_dir}/result.0.nii'], structs=[rigidly_transformed_structure], 
            scale_in_mm=False)

In [ ]:
# Perform the cropping on the planning image, relapse image and the relapse structure set 
create_bounded_image(original_image_path = nii_fixed, 
                     structure_bounds=final_struct_bounds, 
                     saved_image_path = f'{translation_registration_dir}/cropped_{fixed_image}_scan.nii')
create_bounded_image(original_image_path = f'{translation_registration_dir}/result.0.nii',
                     structure_bounds=final_struct_bounds, 
                     saved_image_path = f'{translation_registration_dir}/cropped_{moving_image}_scan.nii')
create_bounded_image(original_image_path = rigidly_transformed_structure,
                     structure_bounds=final_struct_bounds, 
                     saved_image_path = f'{translation_registration_dir}/cropped_{moving_image}_struct.nii')
create_bounded_image(original_image_path = structure_files['fixed_image'],
                     structure_bounds=final_struct_bounds, 
                     saved_image_path = f'{translation_registration_dir}/cropped_{fixed_image}_struct.nii')

In [ ]:
#QuickViewer([f'{translation_registration_dir}/cropped_{fixed_image}_scan.nii', f'{translation_registration_dir}/cropped_{moving_image}_scan.nii'],
           #translation_file_to_overwrite=f'{translation_registration_dir}/TransformParameter_custom.txt', translation=True)
#QuickViewer([f'{translation_registration_dir}/cropped_relapse_scan.nii'], structs= f'{translation_registration_dir}/cropped_relapse_struct.nii')
#QuickViewer([f'{translation_registration_dir}/cropped_planning_scan.nii'])

In [ ]:
#re-regsiter cropped scans?? 
apply_deformable_transform(fixed=f'{translation_registration_dir}/cropped_{fixed_image}_scan.nii', 
                           moving=f'{translation_registration_dir}/cropped_{moving_image}_scan.nii', 
                           registration_dir=translation_registration_dir, 
                           parameter_file=parameter_files_dir,
                           translation_file=out_dir,
                           translation=False)

In [ ]:
# View the re-registered cropped scans 
QuickViewer([f'{translation_registration_dir}/cropped_{fixed_image}_scan.nii', f'{translation_registration_dir}/result.0.nii'], 
          translation_file_to_overwrite=f'{translation_registration_dir}/TransformParameter_custom.txt', translation=True)


In [ ]:
#re-regsiter if changed 
#apply_deformable_transform(fixed=f'{translation_registration_dir}/cropped_{fixed_image}_scan.nii', 
#                           moving=f'{translation_registration_dir}/result.0.nii', 
#                           registration_dir=translation_registration_dir, 
#                           parameter_file=parameter_files_dir,
#                           translation_file=out_dir,
#                           translation=True)

In [ ]:
#QuickViewer([f'{translation_registration_dir}/cropped_{fixed_image}_scan.nii', f'{translation_registration_dir}/result.0.nii'], 
#          translation_file_to_overwrite=f'{translation_registration_dir}/TransformParameter_custom.txt', translation=True)


In [ ]:
# Transform structure set again using cropped transform 
replace_keyword = "FinalNearestNeighborInterpolator"
replace_keywords(out_dir=f'{translation_registration_dir}/TransformParameters.0.txt', 
                search_keyword=search_keyword, 
                replace_keyword=replace_keyword)

rigidly_transformed_structure = apply_rigid_transform(to_transform=f'{translation_registration_dir}/cropped_{moving_image}_struct.nii',
                        transformed_output_dir=f'{translation_registration_dir}/structure_transform_rigid',
                        transform_file=f'{translation_registration_dir}/TransformParameters.0.txt', 
                        transform_type='-in')

# Check the transform of the structure again
QuickViewer([f'{translation_registration_dir}/result.0.nii'], structs=[rigidly_transformed_structure])

In [ ]:
# Check the affine matrix of the two images are the same 
print(nibabel.load(f'{translation_registration_dir}/result.0.nii').affine)
print(nibabel.load(rigidly_transformed_structure).affine)

In [ ]:
parameter_file = 'MI_BSpline10.txt'
parameter_files_dir = ''

registration = os.path.splitext(os.path.basename(parameter_file))[0]
deformable_registration_dir = f'{patient_dir}/{fixed_image}_fixed/{registration}'
if not os.path.isdir(deformable_registration_dir):
    os.makedirs(deformable_registration_dir)

if parameter_files_dir == '':
    voxtox_dir = os.sep.join(voxtox.__file__.split(os.sep)[:-2])
    parameter_files_dir = f'{voxtox_dir}/examples/elastix/elastix_parameter_files/{parameter_file}'

In [ ]:
# Apply B-Spline transform to the cropped translated image 
apply_deformable_transform(fixed=f'{translation_registration_dir}/cropped_{fixed_image}_scan.nii', 
                           moving=f'{translation_registration_dir}/result.0.nii',
                           registration_dir=deformable_registration_dir, 
                           parameter_file=parameter_files_dir,
                           translation_file=out_dir,
                           translation=False)

In [ ]:
# View result of transformed recurrence scan 
QuickViewer([f'{translation_registration_dir}/cropped_{fixed_image}_scan.nii', f'{deformable_registration_dir}/result.0.nii'],
           title=['Cropped Planning Scan', 'Cropped Regsitered Relapse Scan'],
           figsize=5,
           zoom_ui =True,
           comparison=True)

In [ ]:
# Apply deformable registration to cropped and translated thyroid structure set 
search_keyword = "FinalBSplineInterpolator"
replace_keyword = "FinalNearestNeighborInterpolator"
replace_keywords(out_dir=f'{deformable_registration_dir}/TransformParameters.0.txt', 
                search_keyword=search_keyword, 
                replace_keyword=replace_keyword)

deformably_transformed_structure = apply_rigid_transform(to_transform=rigidly_transformed_structure,
                        transformed_output_dir=f'{deformable_registration_dir}/structure_transform_deformable',
                        transform_file=f'{deformable_registration_dir}/TransformParameters.0.txt', 
                        transform_type='-in')

In [ ]:
QuickViewer([f'{deformable_registration_dir}/result.0.nii'], structs=[deformably_transformed_structure])

In [ ]:
QuickViewer([f'{translation_registration_dir}/cropped_{fixed_image}_scan.nii'], 
            structs=[f'{translation_registration_dir}/Croppedplanningthyroid.nii', '/Users/ceilidhwelsh/Documents/CambridgePhD/CodeClub/20210514_TryThisAgain/tmp/VT1_H_F6A233K1/planning_fixed/MI_BSpline10/structure_transform_deformable/TransformedRelapseThyroid.nii'], 
           compare_structs=True, title=['B) Planning Scan'], struct_legend=True, figsize=5.5)

In [ ]:
QuickViewer([f'{translation_registration_dir}/cropped_{fixed_image}_scan.nii'], 
            structs=[f'{translation_registration_dir}/cropped_{fixed_image}_struct.nii', '/Users/ceilidhwelsh/Documents/CambridgePhD/CodeClub/20210514_TryThisAgain/tmp/VT1_H_F6A233K1/planning_fixed/MI_BSpline10/structure_transform_deformable/TransformedRelapseThyroid.nii'], 
           compare_structs=True, title=['A) Planning Scan'], struct_legend=True)
QuickViewer([f'{translation_registration_dir}/cropped_{fixed_image}_scan.nii'], dose=[f'{translation_registration_dir}/dose.nii'],
            structs=[f'{translation_registration_dir}/cropped_{fixed_image}_struct.nii', deformably_transformed_structure], 
           compare_structs=True)

## Extract Dose from within the translated structure set of interest

In [ ]:
def extract_dose_from_nii(dose, path_to_struct_nii, patient, structure_name): 
    """Use dose field to extract the dose from within a selected structure"""
    doseDict={}
    #doseArray = dose.getImageStack()  #this is for the CT object 
    doseArray = dose.get_fdata()       #this passes the dose nii file into the function instead of the CT object 
    
    maskArray = nibabel.load(path_to_struct_nii).get_fdata()
    structureDoseArray = doseArray[maskArray > 0.5]
    doseDict = {f'{structure_name}': list(structureDoseArray)}

    return doseDict  

def get_dose_and_study(patient):
    """Obtains planned dose field for patient"""
    # For patient study check for and obtain the dose files 
    for studyTmp in patient.studyList:
        if studyTmp.doseList:
            study = studyTmp
            dose = study.getPlanDose()
        planning_scan = study.ctList[-1]
    
    return study, dose


def convert_to_nii(file_to_convert, outPath):
    """convert the """
    nifti = voxtox.image.convertCTToNifti(file_to_convert)
    # Create output directory if needed, and write NIfTI object to file
    outPath = voxtox.utility.fullpath(outPath)
    outDirPath = os.path.dirname(outPath)
    if not os.path.exists(outDirPath):
        os.makedirs(outDirPath)
    nifti.to_filename(outPath)
    #convert any file to a nifti file format 
    #currently happens for converting the transformed dose to nii
    #currently happens for converting the transformed structure outline to nii 
    return file_to_convert


study, dose = get_dose_and_study(patient)
dose = voxtox.utility.matchSize(dose, ct[fixed_image], 0)
dose_nii = convert_to_nii(dose, f'{translation_registration_dir}/dose.nii')
cropped_dose_nii = create_bounded_image(original_image_path=f'{translation_registration_dir}/dose.nii',
                                        structure_bounds=final_struct_bounds, 
                                        saved_image_path = f'{translation_registration_dir}/cropped_dose.nii')
cropped_dose_nii = nibabel.load(cropped_dose_nii)


doseDict = extract_dose_from_nii(cropped_dose_nii, deformably_transformed_structure, patient, structure_name='RelapseThyroid')
print(doseDict)

In [ ]:
# Use QuickViewer to visualise the dose field within the transformed structure set 

In [ ]:
deformably_transformed_structure='/Users/ceilidhwelsh/Documents/CambridgePhD/CodeClub/20210514_TryThisAgain/tmp/VT1_H_F6A233K1/planning_fixed/MI_BSpline10/structure_transform_deformable/TransformedRelapseThyroid.nii'

QuickViewer([f'{translation_registration_dir}/cropped_planning_scan.nii'], dose=f'{translation_registration_dir}/cropped_dose.nii',
           structs=[deformably_transformed_structure], colorbar=True, structs_as_mask=True, dose_opacity=1, title=['Dose Within Transformed Relapse Thyroid'],
           struct_legend=True, figsize=5.2)

In [ ]:
QuickViewer([f'{translation_registration_dir}/cropped_planning_scan.nii'], dose=f'{translation_registration_dir}/cropped_dose.nii',
           structs=[deformably_transformed_structure], colorbar=True, title=['Planning Scan With Planned Dose Overlay'], struct_legend=True, figsize=5.2)

In [ ]:
QuickViewer([f'{translation_registration_dir}/cropped_planning_scan.nii'], #dose=f'{translation_registration_dir}/cropped_dose.nii',
          structs=[deformably_transformed_structure], colorbar=True, title=['Planning Scan'], figsize=5.2, struct_legend=True)